In [1]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

In [8]:
# DB

engine = create_engine("sqlite:///private/user_id.db", echo=True)

In [9]:
base = declarative_base()

In [10]:
from sqlalchemy import Column, Integer, String

In [12]:
class User(base):
    __tablename__ = "User"
    
    id = Column(Integer, primary_key=True)
    user_id = Column(Integer)
    username = Column(String)
    first_name = Column(String)
    last_name = Column(String)
    subscribe = Column(Integer)

In [13]:
base.metadata.create_all(engine)

2019-09-12 12:11:02,051 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-09-12 12:11:02,056 INFO sqlalchemy.engine.base.Engine ()
2019-09-12 12:11:02,058 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-09-12 12:11:02,059 INFO sqlalchemy.engine.base.Engine ()
2019-09-12 12:11:02,062 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("User")
2019-09-12 12:11:02,063 INFO sqlalchemy.engine.base.Engine ()
2019-09-12 12:11:02,067 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "User" (
	id INTEGER NOT NULL, 
	user_id INTEGER, 
	username VARCHAR, 
	first_name VARCHAR, 
	last_name VARCHAR, 
	subscribe INTEGER, 
	PRIMARY KEY (id)
)


2019-09-12 12:11:02,068 INFO sqlalchemy.engine.base.Engine ()
2019-09-12 12:11:02,077 INFO sqlalchemy.engine.base.Engine COMMIT


In [15]:
test = User(user_id=12345, username="test", last_name="practice", first_name="fake", subscribe="0")

In [16]:
from sqlalchemy.orm import sessionmaker

In [17]:
Session = sessionmaker(bind=engine)

In [18]:
session = Session()

In [19]:
session.add(test)

In [20]:
session.commit()

2019-09-12 12:11:35,977 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-12 12:11:35,981 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (user_id, username, first_name, last_name, subscribe) VALUES (?, ?, ?, ?, ?)
2019-09-12 12:11:35,982 INFO sqlalchemy.engine.base.Engine (12345, 'test', 'fake', 'practice', '0')
2019-09-12 12:11:35,987 INFO sqlalchemy.engine.base.Engine COMMIT


In [25]:
import telegram
import requests
from bs4 import BeautifulSoup
from telegram.ext import Updater, MessageHandler, Filters, CommandHandler

In [26]:
with open("private/token.txt", 'r') as fp:
    my_token = fp.read()

In [27]:
bot = telegram.Bot(token=my_token)

In [73]:
updater = Updater(token=my_token)

C:\Users\JINHYO\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: TelegramDeprecationWarning: Old Handler API is deprecated - see https://git.io/fxJuV for details
  """Entry point for launching an IPython kernel.


In [74]:
def get_message(bot, update):
    update.message.reply_text("서울특별시빅데이터캠퍼스 공지사항 수신을 하시려면 \"/subscribe 수신\", 공지사항을 수신하지 않으시려면 \"/unsubscribe\"라고 말씀해주세요.\n 이외 기능은 준비 중입니다.")

In [75]:
message_handler = MessageHandler(Filters.text, get_message)

In [76]:
def subscribe_command(bot, update):
    user_id=update.message.chat.id
    username=update.message.chat.username
    last_name=update.message.chat.last_name
    first_name=update.message.chat.first_name
    
    session.add(
        User(
            user_id=user_id,
            username=username,
            last_name=last_name,
            first_name=first_name,
            subscribe="1"
        )
    )
    session.commit()
    
    update.message.reply_text("서울특별시빅데이터캠퍼스 공지사항 수신을 시작합니다.")
    update.message.reply_text(user_id+username+last_name+first_name)

In [77]:
subscribe_command = CommandHandler("subscribe", subscribe_command)

In [78]:
def unsubscribe_command(bot, update):
    session.query.filter_by(user_id=update.message.chat.id).delete()
    update.message.reply_text("서울특별시빅데이터캠퍼스 공지사항 수신을 종료합니다.")

In [79]:
unsubscribe_command = CommandHandler("unsubscribe", unsubscribe_command)

In [80]:
updater.dispatcher.add_handler(message_handler)
updater.dispatcher.add_handler(subscribe_command)
updater.dispatcher.add_handler(unsubscribe_command)

In [81]:
updater.start_polling(timeout=3, clean=True)
updater.idle()

2019-09-12 14:29:18,710 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-12 14:29:18,710 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (user_id, username, first_name, last_name, subscribe) VALUES (?, ?, ?, ?, ?)
2019-09-12 14:29:18,714 INFO sqlalchemy.engine.base.Engine (918578255, None, 'Jinhyo', 'Shin', '1')
2019-09-12 14:29:18,718 INFO sqlalchemy.engine.base.Engine COMMIT
2019-09-12 14:29:59,365 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-09-12 14:29:59,365 INFO sqlalchemy.engine.base.Engine INSERT INTO "User" (user_id, username, first_name, last_name, subscribe) VALUES (?, ?, ?, ?, ?)
2019-09-12 14:29:59,365 INFO sqlalchemy.engine.base.Engine (918578255, None, 'Jinhyo', 'Shin', '1')
2019-09-12 14:29:59,373 INFO sqlalchemy.engine.base.Engine COMMIT
